This project focuses on pruning a deep learning model to make it smaller and more efficient while maintaining accuracy. It applies layer-wise pruning and N:M structured pruning, followed by fine-tuning to recover performance. Initially, the model had 9.9% accuracy, which is very low. This could be due to a few reasons: the model might not be trained properly before pruning, the dataset could be imbalanced, or there might be issues in data preprocessing. After pruning, accuracy remained around 10.38%, showing that pruning alone does not improve performance. However, after fine-tuning, the accuracy jumped to 87.46%, proving that training the pruned model helps regain lost accuracy. The results suggest that pruning works well if applied correctly and fine-tuning is essential to restore model performance.



In [ ]:
# STEP 1: Install Dependencies
!pip install torch torchvision transformers torchsummary datasets tabulate

  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.2.0
    Uninstalling fsspec-2025.2.0:
      Successfully uninstalled fsspec-2025.2.0
ERROR: Operation cancelled by user


In [ ]:
!pip install --upgrade gcsfs fsspec


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0
  Attempting uninstall: gcsfs
    Found existing installation: gcsfs 2024.10.0
    Uninstalling gcsfs-2024.10.0:
      Successfully uninstalled gcsfs-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.2.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2025.2.0 which is incompatible.


In [ ]:
#  STEP 2: Import Libraries
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torchsummary import summary
from tabulate import tabulate

In [ ]:
#  STEP 3: Load Pre-trained Model (ResNet-50 or BERT)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Vision Model (ResNet-50)
model = torchvision.models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 10)  # Change output to 10 classes
model.to(device)

model.eval()
summary(model, (3, 224, 224))

# NLP Model (BERT)
from transformers import BertForSequenceClassification
nlp_model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
nlp_model.to(device)
nlp_model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 128MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
#  STEP 4: Load Dataset (CIFAR-10 for Vision / IMDB for NLP)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))

])

batch_size = 64
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

100%|██████████| 170M/170M [00:04<00:00, 39.2MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
#  STEP 5: Evaluation Function
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = torch.softmax(model(images), dim=1)  # Convert logits to probabilities
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"🔥 Accuracy: {accuracy:.2f}%")
    return accuracy

In [ ]:
#  STEP 6: Test Accuracy Before Pruning
print("🚀 Evaluating model BEFORE pruning:")
accuracy_before = evaluate_model(model, testloader)

🚀 Evaluating model BEFORE pruning:
🔥 Accuracy: 9.90%


In [ ]:
#  STEP 7: Apply Layer-wise Pruning (From Paper [13])
def layerwise_pruning(model, sparsity=0.5):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear) or isinstance(module, torch.nn.Conv2d):
            prune.l1_unstructured(module, name="weight", amount=sparsity)
            prune.remove(module, 'weight')  # Ensure weight tensor is updated


layerwise_pruning(model, sparsity=0.2)  # Reduce sparsity to 20%



In [ ]:
#  STEP 8: Test Accuracy After Layer-wise Pruning
print("🚀 Evaluating model AFTER layer-wise pruning:")
accuracy_after_layerwise = evaluate_model(model, testloader)

🚀 Evaluating model AFTER layer-wise pruning:
🔥 Accuracy: 10.38%


In [ ]:
#  STEP 9: Apply N:M Structured Pruning (From Paper [15])
def structured_pruning(tensor, N=2, M=4):
    shape = tensor.shape
    tensor = tensor.view(-1, M)
    mask = torch.zeros_like(tensor)
    topk_indices = torch.topk(torch.abs(tensor), N, dim=1, largest=True).indices
    mask.scatter_(1, topk_indices, 1)
    return (tensor * mask).view(shape)

for name, param in model.named_parameters():
    if len(param.shape) > 1:
        model.state_dict()[name] = structured_pruning(param, N=2, M=4)

In [ ]:
#  STEP 10: Test Accuracy After N:M Structured Pruning
print("🚀 Evaluating model AFTER N:M structured pruning:")
accuracy_after_nm = evaluate_model(model, testloader)

🚀 Evaluating model AFTER N:M structured pruning:
🔥 Accuracy: 10.38%


In [ ]:
#  STEP 11: Fine-tuning the Pruned Model (Recover Performance)
def fine_tune(model, trainloader, epochs=3, learning_rate=0.001):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}")

fine_tune(model, trainloader, epochs=3)

Epoch 1, Loss: 0.7506
Epoch 2, Loss: 0.4401
Epoch 3, Loss: 0.3368


In [ ]:
#  STEP 12: Test Accuracy After Fine-Tuning
print("🚀 Evaluating model AFTER fine-tuning:")
accuracy_finetuned = evaluate_model(model, testloader)

🚀 Evaluating model AFTER fine-tuning:
🔥 Accuracy: 87.46%


In [ ]:
#  STEP 13: Save the Pruned Model
torch.save(model.state_dict(), "pruned_model.pth")
print(" Pruned model saved successfully!")


In [ ]:
#  STEP 14: Display Comparison Table
data = [
    ["Before Pruning", accuracy_before],
    ["After Layer-wise Pruning", accuracy_after_layerwise],
    ["After N:M Pruning", accuracy_after_nm],
    ["After Fine-Tuning", accuracy_finetuned]
]

print("\n🔥 Performance Comparison:")
print(tabulate(data, headers=["Stage", "Accuracy (%)"], tablefmt="github"))


🔥 Performance Comparison:
| Stage                    |   Accuracy (%) |
|--------------------------|----------------|
| Before Pruning           |           9.9  |
| After Layer-wise Pruning |          10.38 |
| After N:M Pruning        |          10.38 |
| After Fine-Tuning        |          87.46 |
